<a href="https://colab.research.google.com/github/annechris13/Master-Thesis/blob/master/newton_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import pandas as pd
from sklearn.datasets import make_spd_matrix
import random

In [0]:
# from google.colab import drive
# drive.mount("/content/gdrive")

In [0]:
import pandas as pd 
df=pd.read_csv('gdrive/My Drive/test_cases.csv')
# df.head()

In [0]:
# ip=0
# #example problem - 2 batches , 2 variable qp
# nbatch=2
# nBatch=nbatch
# nx=2
# nineq=2
# neq=1
# #to do: extract dimensions from problem parameters + check/add batch dimension
# Q=torch.tensor([[4,1,1,2],[6,2,2,2]]).view(nbatch,nx,nx).type(torch.DoubleTensor)
# p=torch.tensor([[1,1],[1,6]]).view(nbatch,nx).type(torch.DoubleTensor)
# G=torch.tensor([[-1,0,0,-1],[-1,0,0,-1]]).view(nbatch,nineq,nx).type(torch.DoubleTensor)
# h=torch.tensor([[0,0],[0,0]]).view(nbatch,nineq).type(torch.DoubleTensor)
# A=torch.tensor([[1,1],[2,3]]).view(nbatch,neq,nx).type(torch.DoubleTensor)
# b=torch.tensor([[1],[4]]).view(nbatch,neq).type(torch.DoubleTensor)

In [0]:
ip="csv"
nbatch=len(df)
nx=6
nineq=6
neq=3
Q=[]
p=[]
G=[]
h=[]
A=[]
b=[]
for i in range(nbatch):
  seed=int(df["seed"][i])
  random.seed(seed)
  Q.append(make_spd_matrix(nx,random_state=seed).reshape(1,-1))
  p.append([random.randint(0,9) for i in range(nx)])#.astype(float))
  G.append([random.randint(0,9)*((-1)**random.randint(0,1)) for i in range(nineq*nx)])#.astype(float))
  h.append([0 for i in range(nineq)])#.astype(float))
  A.append([random.randint(0,9) for i in range(neq*nx)])#.astype(float))
  b.append([random.randint(0,9) for i in range(neq)])#.astype(float)  )

Q=torch.tensor(Q).view(nbatch,nx,nx).type(torch.DoubleTensor)
p=torch.tensor(p).view(nbatch,nx).type(torch.DoubleTensor)
G=torch.tensor(G).view(nbatch,nineq,nx).type(torch.DoubleTensor)
h=torch.tensor(h).view(nbatch,nineq).type(torch.DoubleTensor)
A=torch.tensor(A).view(nbatch,neq,nx).type(torch.DoubleTensor)
b=torch.tensor(b).view(nbatch,neq).type(torch.DoubleTensor)


In [0]:
#check if Q is psd:
for i in range(nbatch):
  e,_=torch.eig(Q[i])
  if not torch.all(e[:,0]>0):
    raise RuntimeError("Q is not PD")

In [0]:
def lu_hack(x):
    data, pivots = x.lu(pivot=not x.is_cuda)
    if x.is_cuda:
        if x.ndimension() == 2:
            pivots = torch.arange(1, 1+x.size(0)).int().cuda()
        elif x.ndimension() == 3:
            pivots = torch.arange(
                1, 1+x.size(1),
            ).unsqueeze(0).repeat(x.size(0), 1).int().cuda()
        else:
            assert False
    return (data, pivots)

In [0]:
def bdiag(d):
    nBatch, sz, _ = d.size()
    D = torch.zeros(nBatch, sz, sz).type_as(d)
    I = torch.eye(sz).repeat(nBatch, 1, 1).type_as(d).bool()
    D[I] = d.squeeze().view(-1)
    return D

In [0]:
def get_Hessian(Q,G,A):
    nbatch,nineq,nx=G.size()
    neq=A.size()[1]
    B1=torch.zeros(nbatch,nx+nineq,nx+nineq).type_as(Q)
    B3=torch.zeros(nbatch,neq+nineq,nx+nineq).type_as(Q)
    B4=torch.zeros(nbatch,neq+nineq,neq+nineq).type_as(Q)

    B1[:,:nx,:nx]=Q
    B1[:,-nineq:,-nineq:]=torch.eye(nineq).repeat(nbatch,1,1).type_as(Q)

    B3[:,:nineq,:nx]=G
    B3[:,-neq:,:nx]=A
    B3[:,:nineq,nineq:]=torch.eye(nineq).repeat(nbatch,1,1).type_as(Q)

    B2=torch.transpose(B3, dim0=2, dim1=1)

    H=torch.cat((torch.cat((B1,B2),dim=2),torch.cat((B3,B4),dim=2)),dim=1)
  
    return H

In [0]:
def solve_kkt(H,rx,rs,rz,ry,d=None):
    if d!=None:
      D=bdiag(d)
      H[:,nx:nx+nineq,nx:nx+nineq]=D
    # print("H: ",H)
    F=torch.cat((rx,rs,rz,ry), dim=1)
    H_lu,H_piv= lu_hack(H)
    step=F.lu_solve(H_lu,H_piv)

    rx=step[:,:nx,:]
    rs=step[:,nx:nx+nineq,:]
    rz=step[:,nx+nineq:-neq,:]
    ry=step[:,-neq:,:]
    return(rx,rs,rz,ry)

In [0]:
def get_initial(z):
      nbatch,_,_=z.size()
      dz=torch.ones(z.size()).type_as(z)
      alpha=torch.tensor([]).type_as(z)
      for b in range(nbatch):
        step=torch.tensor([-0.1]).type_as(z)
        z_=z[b,:,:]
        dz_=dz[b,:,:]
        while True:
          if (z_+step*dz_ >0).all():
            if step<0:
              alpha=torch.cat((alpha,torch.tensor([0]).type_as(z)))
            else:
              alpha=torch.cat((alpha,1+step))
            break
          else:
            step=step+0.1
      return alpha.view(nbatch,1,1)

In [0]:
def get_step(v,dv):
      #TO DO: find efficient and accurate line search algorithm
      nbatch,_,_=v.size()
      alpha=torch.tensor([]).type_as(v)
      for b in range(nbatch):
        step=torch.tensor([1]).type_as(v)
        v_=v[b,:,:]
        dv_=dv[b,:,:]
        while True:
          if (v_+step*dv_ >=0).all() or step==0:
            alpha=torch.cat((alpha,step))
            break
          else:
            step=step-0.1
      return alpha.view(nbatch,1,1)


In [0]:
def pure_newton(Q,G,A,p,b,h,x,s,z,y, max_iter=20):
    H=get_Hessian(Q,G,A)
    A_T=torch.transpose(A,dim0=2,dim1=1)
    G_T=torch.transpose(G,dim0=2,dim1=1)
    count=0
    for i in range(max_iter):
        rx= -(torch.bmm(A_T,y)+torch.bmm(G_T,z)+torch.bmm(Q,x)+p.unsqueeze(2))
        rs=-z
        rz=-(torch.bmm(G,x)+s-h.unsqueeze(2))
        ry=-(torch.bmm(A,x)-b.unsqueeze(2))
        d=z/s
        #affine step calculation
        dx,ds,dz,dy=solve_kkt(H,rx,rs,rz,ry,d)
        #step size calculation
        alpha = torch.min(get_step(z, dz),get_step(s, ds))
        #update step
        x+=alpha*dx
        s+=alpha*ds
        z+=alpha*dz
        y+=alpha*dy
        mu=torch.abs(torch.bmm(torch.transpose(s,dim0=2,dim1=1),z).sum(1))
        pri_resid=torch.abs(rx)
        dual_1_resid=torch.abs(rz)
        dual_2_resid=torch.abs(ry)
        resids=np.array([pri_resid.max(),mu.max(),dual_1_resid.max(),dual_2_resid.max()])
        if (resids<1e-12).all():
          print("Early exit at iteration no:",i)
          return(x,s,z,y)
        if(i==max_iter-1 and (resids>1e-10).any()):
          print("no of mu not converged: ",len(mu[mu>1e-10]))
          print("no of primal residual not converged: ",len(pri_resid[pri_resid>1e-10]))
          print("no of dual residual 1 not converged: ",len(dual_1_resid[dual_1_resid>1e-10]))
          print("no of dual residual 2 not converged: ",len(dual_2_resid[dual_2_resid>1e-10]))
          raise RuntimeWarning("Residuals not converged, need more itrations")

    return(x,s,z,y)

In [0]:
H=get_Hessian(Q,G,A)
A_T=torch.transpose(A,dim0=2,dim1=1)
G_T=torch.transpose(G,dim0=2,dim1=1)
#initial solution
x,s,z,y=solve_kkt(H,-p.unsqueeze(2),torch.zeros(nbatch,nineq).unsqueeze(2).type_as(Q),h.unsqueeze(2),b.unsqueeze(2))
alpha_p=get_initial(-z)
alpha_d=get_initial(z)
s=-z+alpha_p*(torch.ones(z.size()).type_as(z))
z=z+alpha_d*(torch.ones(z.size()).type_as(z))

In [15]:
x,s,z,y=pure_newton(Q,G,A,p,b,h,x,s,z,y,25)
op_val=0.5*torch.bmm(torch.transpose(x,dim0=2,dim1=1),torch.bmm(Q,x))+torch.bmm(torch.transpose(p.unsqueeze(2),dim0=2,dim1=1),x)

no of mu not converged:  81
no of primal residual not converged:  459
no of dual residual 1 not converged:  438
no of dual residual 2 not converged:  0


RuntimeWarning: ignored

In [16]:
if ip!="csv":
    print("optimal point: \n", x.numpy().reshape(nbatch,-1))
    print("\noptimal objective value: \n", op_val.numpy().reshape(nbatch,-1))

if ip=="csv":
    cols=[str(i)+"nwt" for i in range(nx)]
    error_cp=np.zeros(nbatch)
    error_qp=np.zeros(nbatch)
    new=pd.DataFrame(x.numpy().round(6).reshape(nbatch,-1),columns=cols)
    tmp=df.copy()
    for i,col in enumerate(cols):
      tmp[col]=new[col].copy()
      error_cp+=np.round(tmp[str(i)+"cp"],6)-np.round(tmp[col],6)
      error_qp+=np.round(tmp[str(i)+"qpt"],6)-np.round(tmp[col],6)

    tmp["cp-nwt-error"]=error_cp
    tmp["cp-nwt-er_flag"]=error_cp!=0
    tmp["qpt-nwt-error"]=error_qp
    tmp["qpt-nwt-er_flag"]=error_qp!=0
    print("No.of problems solved: ",len(tmp))
    print("No.of errors from Newton Method: ",len(error_cp[error_cp!=0])," (",round(len(error_cp[error_cp!=0])*100/len(tmp),2),")")
    print("No.of errors from qpth: ",len(tmp[tmp["cp-qpt-er_flag"]])," (",round(len(tmp[tmp["cp-qpt-er_flag"]])*100/len(tmp),2),")")

No.of problems solved:  639
No.of errors from Newton Method:  80  ( 12.52 )
No.of errors from qpth:  22  ( 3.44 )


The newton method produces large no.of inaccurate results (6.4%)(when using a step size decrement of 0.01 (instead of 0.1) in the get_step(), the inaacuracy reduces to just 41 problems(6.42%). This can be associated to the RuntimeWarning that the residuals (and the duality gap mu) has not converged to a value that is close to zero. As long as these values doesnt converge to zero, the results are not optimal! This departure from convergence might be a result of the small steps the algorithm is restricted to take in case of the pure Newton method.

Work around: Add centering component to bias the iterates to move along the central path - path following algorithms - which would help in significant decrease in residuals